In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// This example shows a compact solution of a tandem open queueing network.// Model parametersval lambda = doubleArrayOf(1.0/50.0, 2.0/50.0)  // lambda(r) - arrival rate of class rval D = Matrix.of(arrayOf(    doubleArrayOf(10.0, 5.0),   // D(i,r) - mean demand of class r at station i    doubleArrayOf(5.0, 9.0)))val Z = Matrix.of(doubleArrayOf(91.0, 92.0))  // Z(r) - mean service time of class r at delay stationprintln("\nModel parameters:")println("Lambda: ${lambda.joinToString(", ")}")println("D matrix:")D.print()println("Z: ${Z.toArray1D().joinToString(", ")}")

In [ ]:
// Create tandem PS network with infinite servers
val model = Network.tandemPsInf(Matrix.of(lambda), D, Z)

In [ ]:
// Solve with MVA solver
val solver = MVA(model)
val avgTable = solver.avgTable
avgTable.print()

// 1-line version: AvgTable=MVA(Network.tandemPsInf(lambda,D,Z)).getAvgTable

In [ ]:
// Performance analysis
println("\n=== Performance Analysis ===")

// Class-specific analysis
for (className in arrayOf("Class1", "Class2")) {
    println("\n$className:")
    
    // Extract metrics for this class from JMT results
    var totalRespTime = 0.0
    var totalUtilization = 0.0
    var throughput = 0.0
    
    for (row in 0 until avgTableJmt.getNumRows()) {
        val jobClass = avgTableJmt.get(row, 1).toString()
        if (jobClass == className) {
            val respT = avgTableJmt.get(row, 4) as Double
            val util = avgTableJmt.get(row, 3) as Double
            val tput = avgTableJmt.get(row, 6) as Double
            val station = avgTableJmt.get(row, 0).toString()
            
            totalRespTime += respT
            totalUtilization += util
            if (throughput == 0.0) throughput = tput
            
            if (respT > 0) {
                println("    $station: RespT=${String.format("%.4f", respT)}, Util=${String.format("%.4f", util)}")
            }
        }
    }
    
    println("  Total response time: ${String.format("%.4f", totalRespTime)}")
    println("  Total utilization: ${String.format("%.4f", totalUtilization)}")
    println("  Throughput: ${String.format("%.6f", throughput)}")
}

// System totals
var totalSystemTput = 0.0
for (row in 0 until avgTableJmt.getNumRows()) {
    totalSystemTput += avgTableJmt.get(row, 6) as Double
}
println("\nTotal system throughput: ${String.format("%.6f", totalSystemTput)}")